In [1]:
import sys
import os
import json
import glob
import numpy as np
from lxml import etree
from pyxodr.road_objects.road import Road
from pyxodr.road_objects.lane import Lane,ConnectionPosition,LaneOrientation, TrafficOrientation
from pyxodr.road_objects.junction import Junction
from pyxodr.road_objects.lane_section import LaneSection
from pyxodr.road_objects.network import RoadNetwork
from shapely.geometry import Polygon
from enum import IntEnum


In [89]:
carla_map_dir = 'C:\\Carla-0.10.0\\CarlaUnreal\\Content\\Carla\\Maps\\OpenDrive'
odr_file = os.path.join(carla_map_dir, 'Town04.xodr')

road_network = RoadNetwork(xodr_file_path=odr_file)

In [90]:
roads = road_network.get_roads()
print(f"Number of roads in the network: {len(roads)}")
print(f"Type: {type(roads[0])}\nRoads: {roads}")

Number of roads in the network: 276
Type: <class 'pyxodr.road_objects.road.Road'>
Roads: [Road_0, Road_1, Road_2, Road_3, Road_4, Road_5, Road_6, Road_7, Road_8, Road_9, Road_10, Road_11, Road_12, Road_13, Road_14, Road_15, Road_16, Road_17, Road_18, Road_19, Road_20, Road_21, Road_22, Road_23, Road_24, Road_25, Road_26, Road_27, Road_28, Road_29, Road_30, Road_31, Road_32, Road_33, Road_34, Road_35, Road_36, Road_37, Road_38, Road_39, Road_40, Road_41, Road_42, Road_43, Road_44, Road_45, Road_46, Road_47, Road_48, Road_49, Road_50, Road_51, Road_52, Road_54, Road_56, Road_61, Road_63, Road_65, Road_69, Road_88, Road_89, Road_108, Road_109, Road_110, Road_115, Road_119, Road_123, Road_124, Road_128, Road_135, Road_136, Road_144, Road_145, Road_149, Road_158, Road_161, Road_170, Road_174, Road_179, Road_189, Road_209, Road_210, Road_225, Road_231, Road_239, Road_241, Road_247, Road_254, Road_266, Road_267, Road_271, Road_275, Road_290, Road_291, Road_306, Road_314, Road_319, Road_321, R

In [72]:
multi_lane_section_roads = []

for road in roads:
    if len(road.lane_sections) > 1:
        multi_lane_section_roads.append(road)

print(f"Number of roads with multiple lane sections: {len(multi_lane_section_roads)}")
print("Road IDs with multiple lane sections:")
for road in multi_lane_section_roads:
    print(f"  - {road.id}")

# Check for no links case in lanes of roads with multiple lane sections
for road in multi_lane_section_roads:
    for lane_section in road.lane_sections:
        for lane in lane_section.left_lanes:
            if lane.lane_xml.attrib["type"]=="driving":
                link_xml = lane.lane_xml.find("link")
                if link_xml is None:
                    print(f"  - Road ID: {road.id}, Lane Section ID: {lane_section.lane_section_ordinal}, Lane ID: {lane.id} has no link.")
        for lane in lane_section.right_lanes:
            if lane.lane_xml.attrib["type"]=="driving":
                link_xml = lane.lane_xml.find("link")
                if link_xml is None:
                    print(f"  - Road ID: {road.id}, Lane Section ID: {lane_section.lane_section_ordinal}, Lane ID: {lane.id} has no link.")

Number of roads with multiple lane sections: 10
Road IDs with multiple lane sections:
  - 404
  - 415
  - 438
  - 741
  - 764
  - 799
  - 823
  - 1124
  - 1129
  - 1159


Conclusion: There is no driving lane without a link in a road that has multiple lane_sections

In [93]:
# Finding multi-successor/predecessor roads

multi_successor_roads = []
multi_predecessor_roads = []

for road in road_network.get_roads():
    if len(road.road_xml.find("link").findall("successor")) > 1:
        multi_successor_roads.append(road)
    if len(road.road_xml.find("link").findall("predecessor")) > 1:
        multi_predecessor_roads.append(road)

print(f"Multi-successor roads: {[road.id for road in multi_successor_roads]}")
print(f"Multi-predecessor roads: {[road.id for road in multi_predecessor_roads]}")

# Finding multi-successor/predecessor lanes

multi_successor_lanes = []
multi_predecessor_lanes = []

for road in road_network.get_roads():
    for lane_section in road.lane_sections:
        for lane in lane_section.left_lanes + lane_section.right_lanes:
            if lane.lane_xml.find("link") is not None:
                if lane.lane_xml.find("link").findall("successor") is not []:
                    if len(lane.lane_xml.find("link").findall("successor")) > 1:
                        multi_successor_lanes.append((lane.id, lane_section.lane_section_ordinal, road.id))
                if lane.lane_xml.find("link").findall("predecessor") is not []:
                    if len(lane.lane_xml.find("link").findall("predecessor")) > 1:
                        multi_predecessor_lanes.append((lane.id, lane_section.lane_section_ordinal, road.id))

print(f"Multi-successor lanes: {multi_successor_lanes}")
print(f"Multi-predecessor lanes: {multi_predecessor_lanes}")

Multi-successor roads: []
Multi-predecessor roads: []
Multi-successor lanes: []
Multi-predecessor lanes: []


Conclusion: There is no road element or lane that has multiple successor or predecessor links

In [74]:
def get_road(road_network, road_id):
    for road in road_network.get_roads():
        if road.id == road_id:
            return road
    return None

In [92]:
# Check if every pred of a road has itself as a succ/pred in the pred_road
for road_obj in road_network.get_roads():
    if road_obj.road_xml.attrib["junction"] != "-1":
        continue
    if road_obj.road_xml.find("link") is not None and road_obj.road_xml.find("link").findall("predecessor") is not []:
        for pred in road_obj.road_xml.find("link").findall("predecessor"):
            found = False
            if pred.attrib["elementType"] != "road":
                continue
            pred_road = get_road(road_network, pred.attrib["elementId"])
            if pred_road is not None:
                if pred_road.road_xml.find("link") is not None and pred_road.road_xml.find("link").findall("successor") is not []:
                    for succ in pred_road.road_xml.find("link").findall("successor"):
                        if succ.attrib["elementType"] == "road" and succ.attrib["elementId"] == road_obj.id:
                            found = True
                            break
                if pred_road.road_xml.find("link") is not None and pred_road.road_xml.find("link").findall("predecessor") is not []:
                    for pred_pred in pred_road.road_xml.find("link").findall("predecessor"):
                        if pred_pred.attrib["elementType"] == "road" and pred_pred.attrib["elementId"] == road_obj.id:
                            found = True
                            break
                if not found:
                    print(f"Road {pred_road.id} is a predecessor of {road_obj.id}, but {pred_road.id} does not list {road_obj.id} as a successor/predecessor.")
            else:
                print(f"Predecessor road with ID {pred.attrib['elementId']} not found in the network.")

Conclusion: Every predecessor of a road has itself as a successor/predecessor in the pred_road, and vice-versa for successor

Roads Data Analysis in JSON

In [2]:
json_dir = "../carla"  # replace with your directory path
json_files = glob.glob(os.path.join(json_dir, "*.json"))

for jf in json_files:
    with open(jf, "r") as f:
        print(f"\nReading JSON file: {jf}")
        data = json.load(f)
        if "roads" in data:
            num_geometries = [len(road['geometry']) for road in data['roads']]
            print(f"Number of roads in the JSON: {len(data['roads'])}")
            print(f"Max geometries in a road: {max(num_geometries)}")
            print(f"Min geometries in a road: {min(num_geometries)}")
            print(f"Total geometries in the JSON: {sum(num_geometries)}")
            print(f"Average geometries per road: {sum(num_geometries) / len(num_geometries)}")
        else:
            print("No 'roads' key found in this JSON file.")



Reading JSON file: ../carla\town01.json
Number of roads in the JSON: 346
Max geometries in a road: 3683
Min geometries in a road: 18
Total geometries in the JSON: 47503
Average geometries per road: 137.29190751445086

Reading JSON file: ../carla\town02.json
Number of roads in the JSON: 244
Max geometries in a road: 3015
Min geometries in a road: 15
Total geometries in the JSON: 42614
Average geometries per road: 174.64754098360655

Reading JSON file: ../carla\town03.json
Number of roads in the JSON: 1335
Max geometries in a road: 7856
Min geometries in a road: 2
Total geometries in the JSON: 144513
Average geometries per road: 108.24943820224719

Reading JSON file: ../carla\town04.json
Number of roads in the JSON: 1230
Max geometries in a road: 2982
Min geometries in a road: 2
Total geometries in the JSON: 131327
Average geometries per road: 106.76991869918699

Reading JSON file: ../carla\town05.json
Number of roads in the JSON: 1291
Max geometries in a road: 10000
Min geometries in a